In [1]:
import pandas as pd

dataname = "structured_unstructured_singleterm_zeroshot_gpt-4-turbo-preview"

DATA_PATH = f"../../../Datasets/Evaluations/Sentiment Analysis/Error_Analysis/structured_unstructured/{dataname}.csv"


In [2]:
df = pd.read_csv(DATA_PATH)

In [3]:
df.head()

,Unnamed: 0,text_id,term_id,text,term,polarity,polarity_pred,prompt_name,prompt,error,structured_analysis,unstructured_analysis
0,153,60,153,We had no problems with our resevations and fo...,service,neutral,positive,prompt_3_zeroshot_single_term,We had no problems with our resevations and fo...,error,"[\n{""service"":""well worth a wait"",""importance""...",I made the prediction based on the positive se...
1,291,112,291,My husband and I had dinner here last week and...,dinner,neutral,positive,prompt_3_zeroshot_single_term,My husband and I had dinner here last week and...,error,"[\n{""food"":""0.70""},\n{""good"":""0.30""}\n]","I made the prediction based on the words ""very..."
2,275,105,275,The Scene Shun Lee Palace is popular with midt...,lunch,neutral,positive,prompt_3_zeroshot_single_term,The Scene Shun Lee Palace is popular with midt...,error,"[\n{""upscale room"": ""0.40""},\n{""impress a clie...",I made the prediction based on the positive as...
3,55,19,55,"When I asked the waitress, she told me they co...",salmon,negative,neutral,prompt_3_zeroshot_single_term,"When I asked the waitress, she told me they co...",error,"[\n{""afford"",""importance"":""0.5""},\n{""changed"",...","I determined the sentiment as ""neutral"" based ..."
4,131,50,131,"Next, follow up with another Cosmo and try the...",Hawaiian rib,neutral,positive,prompt_3_zeroshot_single_term,"Next, follow up with another Cosmo and try the...",error,"[\n{""Hawaiian rib eye"": ""0.9""},\n{""delicious"":...",I made my prediction based on the positive con...


In [21]:
print(df['structured_analysis'][1])

[
{"food":"0.70"},
{"good":"0.30"}
]


In [25]:
import pandas as pd
import json
# Function to parse and extract attributes and importance scores
def extract_attributes(s):
    try:
        # Parse the JSON string
        parsed = json.loads(s)
        # Extract attributes and importance scores
        return [(dct[list(dct.keys())[0]], list(dct.keys())[0]) for dct in parsed]
    except (json.JSONDecodeError, TypeError):
        # Return None or an empty list if there's a parsing error or unexpected format
        return None

# Apply the function to each row and expand the lists into a new dataframe
attributes_list = df['structured_analysis'].apply(extract_attributes)

# Initialize lists to hold the extracted data
attributes = []
importances = []

# Loop through the lists, adding data to the attributes and importances lists
for item in attributes_list:
    if item is not None:  # Only proceed if the item is not None
        for importance, attribute in item:
            attributes.append(attribute)
            importances.append(importance)

# Create the new dataframe
df_values = pd.DataFrame({
    'attribute': attributes,
    'importance': importances
})

print(df_values)

            attribute         importance
0             service  well worth a wait
1                food               0.70
2                good               0.30
3        upscale room               0.40
4    impress a client               0.30
..                ...                ...
231     dissapointing                0.2
232             chole                0.4
233          delicate                0.4
234            subtle                0.4
235           dessert                0.2

[236 rows x 2 columns]


In [27]:
import pandas as pd
import numpy as np

# Assuming df_values is already created and cleaned as per the previous steps
# Example df_values for illustration (cleaned version)
df_values = pd.DataFrame({
    'attribute': ['food', 'good', 'service', 'quality', 'food'],
    'importance': [0.70, 0.30, 0.80, 0.20, 0.75]
})

# Group by attribute and calculate required statistics
summary_df = df_values.groupby('attribute').agg(
    occurrence_count=('attribute', 'size'),
    average_importance=('importance', 'mean'),
    std_importance=('importance', 'std')
).reset_index()

# Replace NaN in std_importance with 0 (occurs when there's only one occurrence of an attribute)
summary_df['std_importance'].fillna(0, inplace=True)

# Sort by occurrence count in descending order
summary_df = summary_df.sort_values(by='occurrence_count', ascending=False).reset_index(drop=True)

print(summary_df)


  attribute  occurrence_count  average_importance  std_importance
0      food                 2               0.725        0.035355
1      good                 1               0.300        0.000000
2   quality                 1               0.200        0.000000
3   service                 1               0.800        0.000000


/var/folders/pw/9fnphybj4kq8t6wt3h592_9r0000gn/T/ipykernel_2313/2171538453.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  summary_df['std_importance'].fillna(0, inplace=True)


In [28]:
# Calculate overall mean, std, and var of importance scores
overall_mean = df_values['importance'].mean()
overall_std = df_values['importance'].std()
overall_var = df_values['importance'].var()

print(f"Overall Mean Importance: {overall_mean}")
print(f"Overall Standard Deviation of Importance: {overall_std}")
print(f"Overall Variance of Importance: {overall_var}")



Overall Mean Importance: 0.55
Overall Standard Deviation of Importance: 0.2783882181415011
Overall Variance of Importance: 0.0775
